In [5]:
import datetime
import numpy as np
import talib as ta
import alpaca_trade_api as tradeapi
import time
import sys
sys.path.insert(0, '../')


from datetime import timedelta

In [6]:
from config_paper import key_id, secret_key
from cross_signal import *

In [7]:
#The url at the end is what makes a paper trading bot
base_url='https://paper-api.alpaca.markets'
api = tradeapi.REST(key_id, secret_key,base_url )

In [4]:
today = datetime.datetime.today().date() # or datetime.now to use local timezone
daystart=pd.Timestamp(year=today.year, month=today.month, day=today.day, hour=0,tz='America/New_York').isoformat()

In [5]:
#make sure to import these packages
from pandas import DataFrame
import talib.abstract as tab
#abstract automatically extracts the columns and periods from the data itself, no input required



#feel free to add or subtract more indicators for any other strategy you want to use
def populateindicators(dataframe) -> DataFrame:
        #exponential moving averages
        dataframe['ema_high'] = tab.EMA(dataframe, timeperiod=10, price='high')
        dataframe['ema_close'] = tab.EMA(dataframe, timeperiod=10, price='close')
        dataframe['ema_low'] = tab.EMA(dataframe, timeperiod=10, price='low')
        
        
        dataframe['mfi'] = ta.MFI(dataframe['high'],dataframe['low'],dataframe['close'], np.asarray(dataframe['volume'], dtype='float'), timeperiod=14)
        dataframe['ultosc'] = ta.ULTOSC(dataframe['high'],dataframe['low'],dataframe['close'])
        dataframe['adosc'] = ta.ADOSC(dataframe['high'],dataframe['low'],dataframe['close'], np.asarray(dataframe['volume'], dtype='float'))
        
        
        dataframe['minusdi'] = tab.MINUS_DI(dataframe, timeperiod=14)
        dataframe['plusdi'] = tab.PLUS_DI(dataframe, timeperiod=14)
        
        dataframe['rsi'] = tab.RSI(dataframe, timeperiod=20, price='close')
        stoch_fast = tab.STOCHF(dataframe, 10.0, 3.0, 0.0, 3.0, 0.0)
        dataframe['fastd'] = stoch_fast['fastd']
        dataframe['fastk'] = stoch_fast['fastk']
        dataframe['adx'] = tab.ADX(dataframe)
        dataframe['cci'] = tab.CCI(dataframe)
        # required for graphing

        return dataframe

In [6]:
account = api.get_account()
print(account.status)
account.buying_power

ACTIVE


'209712.5'

In [7]:
with open('SP500_2020Q2SYM.txt', 'r') as f:
    symbols= f.read().split('\n')

In [8]:
barTimeframe = "1Min" # 1Min, 5Min, 15Min, 1H, 1D
assetsToTrade = [x.upper() for x in symbols]
positionSizing = 0.01

# Tracks position in list of symbols to download 
assetListLen = len(assetsToTrade)
trade=True

# Bar get with alpaca

In [ ]:
trade=True
while trade==True:
    #asset Manager

    assetlist=api.list_positions()
    for asset in assetlist:
            #if asset is in list of positions AND is more than 2.5% loss
            # sell and turn that symbol to null
        if float(asset.unrealized_intraday_plpc)< -.023 :
            openPosition = api.get_position(symbol)

            returned = api.submit_order(symbol,int(openPosition.qty),"sell","market","day")
            assetsToTrade[i]= symbol+' nulled for below -2.5% pnl'
            print('Sell', symbol)    
            
            
    for i in range(assetListLen) :
        try:
            symbol = assetsToTrade[i]

            returned_data = api.get_barset(symbol,barTimeframe,limit=150,start=daystart).df
            #api.get_barset(symbol,barTimeframe,limit=150).df


            # Calculated trading indicators
            #populate exponential moving averages
            data=populateindicators(returned_data) 

            buyord=False
            
                    
            
            
             # Create the buy signal here
            if data['adx'][-1] > 22 and \
            data['mfi'][-1] < 25 and \
            data['ultosc'][-1] < 25 and \
            data['rsi'][-1] < 33 and \
            data['minusdi'][-1] > 25 and \
            data['minusdi'][-1] > data['plusdi'][-1]:

                buyord=True
                assetlist=api.list_positions()
                for asset in assetlist:
                    # if asset is in the list of positions we have already bought, then do not buy again
                    if asset.symbol == symbol:
                        buyord=False
                        print(asset.symbol,asset.qty)
                    
                    
                    
                      
                            
                #if asset is not already bought, then buy             
                if buyord==True:
                    #cashBalance = account.daytrading_buying_power
                    cashBalance = float(account.buying_power)

                    targetPositionSize = int(cashBalance / (data['close'][-1] / positionSizing))

                    returned = api.submit_order(symbol,targetPositionSize,"buy","market","day") # Market order to open position
                    print('Buy ',returned)

       #Sell conditions 
            elif data['adx'][-1] > 22 and \
                data['rsi'][-1] > 75:
                # Closes position
                openPosition = api.get_position(symbol)

                returned = api.submit_order(symbol,int(openPosition.qty),"sell","market","day") # Market order to fully close position
                print('Sell',returned)
        

        except:
                print(symbol)
    
    
    #get current time and shut off trading
    currenthour=time.localtime().tm_hour 
    if currenthour >= 12: 
        trade=False
        print('Good work, Turning off')
        
    #interval number of seconds to wait
    time.sleep(5)
    
    